In [2]:
using Graphs, Graphs.Experimental.ShortestPaths
using GraphIO

In [3]:
using GraphIO.NET
using Statistics
using DataFrames

file_path = "A1-networks/toy/wheel.net"
graph = loadgraph(file_path, "graph_key", NETFormat())

{9, 16} undirected simple Int64 graph

In [29]:
function read_weighted_net(file_path)
    graph = loadgraph(file_path, "graph_key", NETFormat())
    
    edge_weights = Dict{Tuple{Int,Int}, Float64}()
    open(file_path, "r") do file
        reading_edges = false
        for line in eachline(file)
            if occursin("*Edges", line)
                reading_edges = true
            elseif reading_edges
                edge_data = split(line)
                if length(edge_data) >= 3
                    src, dst, weight = parse(Int, edge_data[1]), parse(Int, edge_data[2]), parse(Float64, edge_data[3])
                    edge_weights[(src, dst)] = weight
                    # Assuming you add edges to your graph here
                    add_edge!(graph, src, dst)  # Add the edge to the graph if not already added
                end
            end
        end
    end
    return graph, edge_weights
end

read_weighted_net (generic function with 1 method)

In [30]:
graph, edge_weights = read_weighted_net("A1-networks/toy/wheel.net")

(SimpleGraph{Int64}(16, [[2, 3, 4, 5, 6, 7, 8, 9], [1, 3, 9], [1, 2, 4], [1, 3, 5], [1, 4, 6], [1, 5, 7], [1, 6, 8], [1, 7, 9], [1, 2, 8]]), Dict((4, 5) => 1.0, (1, 2) => 1.0, (1, 3) => 1.0, (1, 4) => 1.0, (9, 2) => 1.0, (7, 8) => 1.0, (8, 9) => 1.0, (1, 9) => 1.0, (3, 4) => 1.0, (5, 6) => 1.0…))

In [44]:
function network_num_descriptors(graph, verbose = true)
    # Number of nodes and edges
    num_nodes = nv(graph)
    num_edges = ne(graph)

    # Degrees
    degrees = degree(graph)
    min_degree = minimum(degrees)
    max_degree = maximum(degrees)
    avg_degree = mean(degrees)

    # Average clustering coefficient 
    avg_clustering_coefficient = sum(local_clustering_coefficient(graph, vertices(graph)))/nv(graph)

    # Assortativity 
    assort = assortativity(graph)

    # Average path length and Diameter
    # For a large graph, this might be computationally expensive
    avg_path_length = sum(shortest_paths(graph).dists)/(num_nodes*(num_nodes-1))
    diam = Graphs.diameter(graph)  # Same adjustment as above
    
    if verbose
        # Printing the results
        println("Number of nodes: $num_nodes")
        println("Number of edges: $num_edges")
        println("Degree -- Min: $min_degree, Max: $max_degree, Avg: $avg_degree")
        println("Average Clustering Coefficient: $avg_clustering_coefficient")
        println("Assortativity: $assort")
        println("Average Path Length: $avg_path_length")
        println("Diameter: $diam")
    end
    
    return [isa(i, AbstractFloat) ? round(i, digits=4) : i for i in [num_nodes, num_edges, min_degree, max_degree, avg_degree, avg_clustering_coefficient, assort, avg_path_length, diam]]
end

network_num_descriptors (generic function with 2 methods)

In [46]:
network_num_descriptors(graph)

Number of nodes: 9
Number of edges: 16
Degree -- Min: 3, Max: 8, Avg: 3.5555555555555554
Average Clustering Coefficient: 0.6243386243386244
Assortativity: -0.3333333333333333
Average Path Length: 1.5555555555555556
Diameter: 2


9-element Vector{Float64}:
  9.0
 16.0
  3.0
  8.0
  3.5556
  0.6243
 -0.3333
  1.5556
  2.0

In [13]:
function construct_df(data_dict, col_names = [])
    # Initialize an empty DataFrame with appropriate column names
    if isempty(col_names)
        col_names = ["Col$(i)" for i in 1:length(first(values(data_dict)))]
    end
    
    col_names_with_key = ["Index"; col_names]
    
    # Initialize an empty DataFrame with the updated column names
    df = DataFrame(; (Symbol(col_name) => Any[] for col_name in col_names_with_key)...)
    
    # Sort the dictionary by keys to maintain the order
    sorted_keys = sort(collect(keys(data_dict)))

    # Populate the DataFrame, including the key as the first column in each row
    for key in sorted_keys
        row_data = [key; data_dict[key]]
        push!(df, row_data)
    end
    
    return df
end

construct_df (generic function with 2 methods)

In [36]:
function compute_strength(edge_weights, mode="undirected")
    weights_count = Dict{Int, AbstractFloat}()
    if mode == "undirected"
        for (key,value) in edge_weights
            for v in key
                if haskey(weights_count, v)
                    weights_count[v] += value
                else
                    weights_count[v] = value
                end
            end
        end
    elseif mode == "directed"
        for (key,value) in edge_weights
            v = key[1]
            if haskey(weights_count, v)
                weights_count[v] += value
            else
                weights_count[v] = value
            end
        end
    end
    return weights_count
end

compute_strength (generic function with 2 methods)

In [4]:
function nodes_num_descriptors(graph, edge_weights, verbose = false)
    num_nodes = nv(graph)
    node_descriptors = Dict{Int, Vector}()
    degrees = degree(graph)
    strenghts = compute_strength(edge_weights)
    aspls = shortest_paths(graph).dists
    lspls = maximum(collect(adjacency_matrix(graph)); dims = 2)
    b_centralities = betweenness_centrality(graph) 
    e_centralities = eigenvector_centrality(graph)
    pageranks = pagerank(graph)
    for i in vertices(graph)
        deg = degrees[i]
        stren = strenghts[i]
        aspl = sum(aspls[i,:])/(num_nodes-1)
        lspl = lspls[i]
        cl_cf = local_clustering_coefficient(graph, i)
        bc = b_centralities[i] 
        ec = e_centralities[i]
        pr = pageranks[i]
        
        node_descriptors[i] = [deg,stren, aspl, lspl, cl_cf, bc, ec, pr]
    end
    
    df_nodes_descriptors = construct_df(node_descriptors, ["Degree", "Strength", "ASPL", "LSPL", "Clust Coeff", "Betweeness", "Eigenvector", "PageRank"])
    
    if verbose
        println(df_nodes_descriptors)
    end
    
    return df_modes_descriptors
end

nodes_num_descriptors (generic function with 1 method)

In [176]:
nodes_num_descriptors(graph)

9×8 DataFrame
 Row │ Degree   Strength  ASPL     LSPL     Clust Coeff  Betweeness  Eigenvector  PageRank  
     │ Float64  Float64   Float64  Float64  Float64      Float64     Float64      Float64   
─────┼──────────────────────────────────────────────────────────────────────────────────────
   1 │     8.0       8.0    1.0        1.0     0.285714   0.571429      0.57735   0.233766
   2 │     3.0       3.0    1.625      1.0     0.666667   0.0178571     0.288675  0.0957793
   3 │     3.0       3.0    1.625      1.0     0.666667   0.0178571     0.288675  0.0957793
   4 │     3.0       3.0    1.625      1.0     0.666667   0.0178571     0.288675  0.0957793
   5 │     3.0       3.0    1.625      1.0     0.666667   0.0178571     0.288675  0.0957793
   6 │     3.0       3.0    1.625      1.0     0.666667   0.0178571     0.288675  0.0957793
   7 │     3.0       3.0    1.625      1.0     0.666667   0.0178571     0.288675  0.0957793
   8 │     3.0       3.0    1.625      1.0     0.666667   0.0178

## Final Tests

In [1]:
include("../Assignment 1/NetworkProcessing.jl")

Main.NetworkProcessing

In [2]:
graph, edge_weights = NetworkProcessing.read_network("A1-networks/real/airports_UW.net")
NetworkProcessing.network_num_descriptors(graph)

Number of nodes: 3618
Number of edges: 14142
Degree -- Min: 1, Max: 250, Avg: 7.817578772802653
Average Clustering Coefficient: 0.4957489312349576
Assortativity: 0.04622413053190826
Average Path Length: 4.439594641910406
Diameter: 17


9-element Vector{Float64}:
  3618.0
 14142.0
     1.0
   250.0
     7.8176
     0.4957
     0.0462
     4.4396
    17.0

In [4]:
NetworkProcessing.nodes_num_descriptors(graph, edge_weights)

Row,Index,Degree,Strength,ASPL,LSPL,Clust Coeff,Betweeness,Eigenvector,PageRank
,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,1.0,2.0,92.0,4.517,1.0,1.0,0.0,8.966e-5,0.00014614
2,2.0,6.0,7284.0,3.67763,1.0,0.733333,6.389e-5,0.00591853,0.00018338
3,3.0,3.0,13071.0,3.91042,1.0,1.0,0.0,0.00332087,0.00010462
4,4.0,11.0,3224.5,3.92784,1.0,0.672727,1.012e-5,0.0069863,0.00027764
5,5.0,10.0,1690.0,3.81006,1.0,0.333333,2.003e-5,0.00656959,0.0002747
6,6.0,6.0,11653.0,3.60077,1.0,0.933333,2.0e-8,0.00911095,0.00016614
7,7.0,1.0,28.5,4.68261,1.0,0.0,0.0,0.00012477,6.949e-5
8,8.0,1.0,38.0,5.63202,1.0,0.0,0.0,8.6e-7,0.00010724
9,9.0,4.0,646.0,4.51783,1.0,0.833333,6.0e-8,0.0005958,0.00014285


In [3]:
using CSV

net_descriptors = Dict{String, Vector}()

for file in readdir("NatNets")
    graph, edge_weights, vertex_labels  = NetworkProcessing.read_network("NatNets/$file")
   net_descriptors[file] = NetworkProcessing.network_num_descriptors(graph, false)
end


┌ Warning: Infinite path length detected for vertex 1
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 2
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 3
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 4
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 5
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 6
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 7
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 8
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\F

┌ Warning: Infinite path length detected for vertex 53
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 54
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 55
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 56
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 57
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 58
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 59
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 60
└ @ Graphs C:\Users\mario\.julia\packages\

┌ Warning: Infinite path length detected for vertex 105
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 106
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 107
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 108
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 109
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 110
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 111
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 112
└ @ Graphs C:\Users\mario\.julia\p

┌ Warning: Infinite path length detected for vertex 156
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 157
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 158
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 159
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 160
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 161
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 162
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 163
└ @ Graphs C:\Users\mario\.julia\p

┌ Warning: Infinite path length detected for vertex 207
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 208
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 209
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 210
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 211
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 212
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 213
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 214
└ @ Graphs C:\Users\mario\.julia\p

┌ Warning: Infinite path length detected for vertex 258
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 259
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 260
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 261
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 262
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 263
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 264
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 265
└ @ Graphs C:\Users\mario\.julia\p

┌ Warning: Infinite path length detected for vertex 310
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 311
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 312
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 313
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 314
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 315
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 316
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 317
└ @ Graphs C:\Users\mario\.julia\p

┌ Warning: Infinite path length detected for vertex 354
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 355
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 356
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 357
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 358
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 359
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 360
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 361
└ @ Graphs C:\Users\mario\.julia\p

┌ Warning: Infinite path length detected for vertex 405
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 406
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 407
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 408
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 409
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 410
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 411
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 412
└ @ Graphs C:\Users\mario\.julia\p

┌ Warning: Infinite path length detected for vertex 456
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 457
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 458
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 459
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 460
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 461
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 462
└ @ Graphs C:\Users\mario\.julia\packages\Graphs\FXxqo\src\distance.jl:78
┌ Warning: Infinite path length detected for vertex 463
└ @ Graphs C:\Users\mario\.julia\p

In [4]:
df = NetworkProcessing.construct_df(net_descriptors, ["# Nodes", "# Edged", "Degree (min)", "Degree (max)", "Degree (avg)", "ACC", "Assort.", "APL", "Diameter"])
# CSV.write("ExerciseA_Results.csv", df)
df

Row,Index,# Nodes,# Edged,Degree (min),Degree (max),Degree (avg),ACC,Assort.,APL,Diameter
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,ER_500_05.net,500.0,62203.0,215.0,281.0,248.812,0.4987,-0.0073,1.5014,2.0
2,ER_500_07.net,500.0,87163.0,315.0,377.0,348.652,0.6987,-0.0034,1.3013,2.0
3,ER_N500_k6.net,499.0,1447.0,0.0,14.0,5.7996,0.0119,0.0323,3.7031,9.22337e18
4,Erdos-Renyi_500_0.3.net,500.0,37579.0,115.0,182.0,150.316,0.3008,0.0014,1.6988,2.0
5,Erdos-Renyi_500_0.7.net,500.0,87563.0,313.0,380.0,350.252,0.7019,-0.0058,1.2981,2.0
6,Erdos-Renyi_700_0.3.net,700.0,73519.0,166.0,245.0,210.054,0.3006,-0.0064,1.6995,2.0
7,Erdos-Renyi_700_0.7.net,700.0,171238.0,452.0,526.0,489.251,0.6999,-0.004,1.3001,2.0


In [4]:
graph, edge_weights, vertex_labels = NetworkProcessing.read_network("A1-networks/real/airports_UW.net")
NetworkProcessing.nodes_num_descriptors(graph, edge_weights, vertex_labels)

Row,Index,Degree,Strength,ASPL,LPL,Clust Coeff,Betweeness,Eigenvector,PageRank
,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,AAA,2.0,92.0,4.517,12.0,1.0,0.0,8.966e-5,0.00014614
2,AAE,6.0,7284.0,3.67763,11.0,0.733333,6.389e-5,0.00591853,0.00018338
3,AAL,3.0,13071.0,3.91042,11.0,1.0,0.0,0.00332087,0.00010462
4,AAN,11.0,3224.5,3.92784,11.0,0.672727,1.012e-5,0.0069863,0.00027764
5,AAQ,10.0,1690.0,3.81006,11.0,0.333333,2.003e-5,0.00656959,0.0002747
6,AAR,6.0,11653.0,3.60077,11.0,0.933333,2.0e-8,0.00911095,0.00016614
7,AAT,1.0,28.5,4.68261,12.0,0.0,0.0,0.00012477,6.949e-5
8,AAU,1.0,38.0,5.63202,13.0,0.0,0.0,8.6e-7,0.00010724
9,AAY,4.0,646.0,4.51783,12.0,0.833333,6.0e-8,0.0005958,0.00014285
